## The object of this project is to explore, segment, and cluster the neighborhoods in the city of Toronto. From the analysis, we can obtain the insight that what regions in Toronto are similar to each other and different from the rest in terms of some common or distinguishable features. 
### The project includes three parts:
#### - Part 1 (in a separated Notebook) involves scraping and cleasing the data of the basic Toronto neighborhood information. 
#### - Part 2 (in a separated Notebook) continues with the cleaned neighborhood data and uses Foursquare API to retrieve more details around each neighborhood, which are considered features and needed for further analysis. 
#### - In Part 3, the features are transformed so that segmenting and clustering can be applied. Folium map module is used here for geographical visualization. K-means clustering through Scikit-learn is used here for the model, and insight is found and analyzed in this section.  

## Part 3: explore and cluster the neighbourhoods and find insights

In [79]:
# import necessary modules 
import numpy as np
import pandas as pd

import requests

# import folium maps module
#!conda install -c conda-forge folium=0.5.0 --yes  
import folium 

# import modules needed for geo location conversion
#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt

# import k-means from clustering stage
from sklearn.cluster import KMeans

Read the cleaned and grouped data with coordinates organized from Part 1 & 2 

In [126]:
url_cleaned_data = 'https://raw.githubusercontent.com/liangyiteng888/Coursera_Capstone/master/Toronto_Neighborhood_with_Coordinates.csv'
df_neighbor_toronto = pd.read_csv(url_cleaned_data, index_col = 0)
#help(pd.read_csv)

Uncomment the selection part if only wanting to Work on boroughs that contain the word 'Toronto'. Extract matching rows from the dataframe.

In [127]:
df_neighbor_toronto = df_neighbor_toronto[['Toronto' in i for i in df_neighbor_toronto['Borough']]]
df_neighbor_toronto.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


Reorganize the dataframe so that same Borough rows gather together 

In [128]:
df_neighbor_toronto = df_neighbor_toronto.sort_values('Borough').reset_index(drop=True)
df_neighbor_toronto.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M5R,Central Toronto,"The Annex, North Midtown, Yorkville",43.672710,-79.405678
1,M5P,Central Toronto,"Forest Hill North, Forest Hill West",43.696948,-79.411307
2,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
3,M4P,Central Toronto,Davisville North,43.712751,-79.390197
4,M4R,Central Toronto,North Toronto West,43.715383,-79.405678


Create a map of Toronto with neighborhoods superimposed on top. Use **Folium** map rendering library for the map creation and manipulation.

In [59]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [60]:
# create map of Toronto  using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_neighbor_toronto['Latitude'], df_neighbor_toronto['Longitude'], df_neighbor_toronto['Borough'], df_neighbor_toronto['Neighborhood']):
    label = '{} ({})'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.5,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Next, going to start utilizing the Foursquare API to explore the neighborhoods and segment them.
#### Define Foursquare Credentials and Version

In [61]:
CLIENT_ID = 'UFVBX0AITYQGIOHCTKAXCAOREBEZYWFR55WFJZ1DDAEZ53L2' # your Foursquare ID
CLIENT_SECRET = 'BV20PKRLNZGAQRMFGOIRNXE4MTBQ3NENGIRN01DXIR5PC3UE' # your Foursquare Secret
VERSION = '20200220' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: UFVBX0AITYQGIOHCTKAXCAOREBEZYWFR55WFJZ1DDAEZ53L2
CLIENT_SECRET:BV20PKRLNZGAQRMFGOIRNXE4MTBQ3NENGIRN01DXIR5PC3UE


In [62]:
neighborhood_latitude = df_neighbor_toronto.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_neighbor_toronto.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = df_neighbor_toronto.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Moore Park, Summerhill East are 43.6895743, -79.38315990000002.


#### Now, get the top 100 venues that are in The Annex, North Midtown, Yorkville within a radius of 500 meters.
First, let's create the GET request URL. Name your URL **url**.

In [63]:
LIMIT= 100
radius = 500
url =  'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=UFVBX0AITYQGIOHCTKAXCAOREBEZYWFR55WFJZ1DDAEZ53L2&client_secret=BV20PKRLNZGAQRMFGOIRNXE4MTBQ3NENGIRN01DXIR5PC3UE&v=20200220&ll=43.6895743,-79.38315990000002&radius=500&limit=100'

In [64]:
results = requests.get(url).json()

In [65]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [66]:
venues = results['response']['groups'][0]['items']
#print(venues)
nearby_venues = json_normalize(venues) # flatten JSON
#print(nearby_venues.head())
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Ravine,Trail,43.690356,-79.386841
1,Reds,Restaurant,43.692816,-79.384504
2,Moore Park Tennis Club,Tennis Court,43.693289,-79.382900
3,Moorevale Park,Playground,43.693610,-79.383465


In [67]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

4 venues were returned by Foursquare.


### Part 3.1 Explore Neighborhoods in Toronto

Create a function to repeat the same process to all the neighborhoods in Toronto

In [68]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [69]:
Toronto_venues = getNearbyVenues(names=df_neighbor_toronto['Neighborhood'],
                                   latitudes=df_neighbor_toronto['Latitude'],
                                   longitudes=df_neighbor_toronto['Longitude']
                                  )

Moore Park, Summerhill East
Roselawn
Davisville
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville
Lawrence Park
Davisville North
North Toronto West
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Design Exchange, Toronto Dominion Centre
St. James Town
Rosedale
Church and Wellesley
Harbourfront
Ryerson, Garden District
Berczy Park
Adelaide, King, Richmond
Central Bay Street
Commerce Court, Victoria Hotel
Harbord, University of Toronto
Queen's Park
Christie
Harbourfront East, Toronto Islands, Union Station
Stn A PO Boxes 25 The Esplanade
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
First Canadian Place, Underground city
Chinatown, Grange Park, Kensington Market
Cabbagetown, St. James Town
The Beaches
The Beaches West, India Bazaar
The Danforth West, Riverdale
Business Reply Mail Processing Centre 969 Eastern
Studio District
Parkdale, Roncesvalles
Brockton, Exhibition Place, Parkdale Vill

#### Drop records where the 'Venue Category' is indicated as 'Neighborhood'

In [70]:
print(Toronto_venues.shape)
Toronto_venues = Toronto_venues.drop(Toronto_venues[['Neighborhood' in i for i in Toronto_venues['Venue Category']]].index).reset_index(drop = True)

(1731, 7)


Show the number of venues obtained for each of the neighborhood

In [71]:
Toronto_venues.groupby('Neighborhood')['Venue'].count().reset_index(name = 'Number of venues')

,Neighborhood,Number of venues
0,"Adelaide, King, Richmond",99
1,Berczy Park,57
2,"Brockton, Exhibition Place, Parkdale Village",23
3,Business Reply Mail Processing Centre 969 Eastern,18
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",17
5,"Cabbagetown, St. James Town",49
6,Central Bay Street,79
7,"Chinatown, Grange Park, Kensington Market",87
8,Christie,18
9,Church and Wellesley,86


In [72]:
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))

There are 237 uniques categories.


### Part 3.2 Transform the venue categories for each neighborhood to be applicable for clustering 

Used one hot encoding to transform the venue categories into 0/1 numbers needed as input for the clustering modelling

In [73]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]
#print(fixed_columns)
print(Toronto_onehot.shape)
Toronto_onehot.head()

(1727, 238)


,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Moore Park, Summerhill East",0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,"Moore Park, Summerhill East",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Moore Park, Summerhill East",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Moore Park, Summerhill East",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Roselawn,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [74]:
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
print(Toronto_grouped.shape)
Toronto_grouped

(39, 238)


,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020202,0.000000,0.000000,...,0.00000,0.000000,0.020202,0.000000,0.000000,0.010101,0.000000,0.000000,0.010101,0.000000
1,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.017544,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Brockton, Exhibition Place, Parkdale Village",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Business Reply Mail Processing Centre 969 Eastern,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.055556
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.000000,0.058824,0.058824,0.117647,0.176471,0.117647,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,"Cabbagetown, St. James Town",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020408,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012658,0.000000,0.000000,...,0.00000,0.000000,0.012658,0.000000,0.000000,0.012658,0.000000,0.000000,0.000000,0.012658
7,"Chinatown, Grange Park, Kensington Market",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.034483,0.000000,0.057471,0.011494,0.000000,0.000000,0.000000,0.000000
8,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Church and Wellesley,0.011628,0.000000,0.000000,0.000000,0.000000,0.000000,0.011628,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.011628,0.000000,0.000000,0.011628,0.000000,0.011628


#### Print each neighborhood along with the top 3 most common venues

In [75]:
num_top_venues = 3

for row in Toronto_grouped['Neighborhood']:
    print("----"+row+"----")
    temp = Toronto_grouped[Toronto_grouped['Neighborhood'] == row].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
         venue  freq
0  Coffee Shop  0.07
1         Café  0.04
2   Restaurant  0.04


----Berczy Park----
               venue  freq
0        Coffee Shop  0.07
1  French Restaurant  0.04
2       Cocktail Bar  0.04


----Brockton, Exhibition Place, Parkdale Village----
            venue  freq
0            Café  0.13
1  Breakfast Spot  0.09
2     Coffee Shop  0.09


----Business Reply Mail Processing Centre 969 Eastern----
                  venue  freq
0    Light Rail Station  0.11
1  Gym / Fitness Center  0.06
2               Brewery  0.06


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
              venue  freq
0   Airport Service  0.18
1    Airport Lounge  0.12
2  Airport Terminal  0.12


----Cabbagetown, St. James Town----
         venue  freq
0  Coffee Shop  0.08
1  Pizza Place  0.06
2       Bakery  0.06


----Central Bay Street----
                venue  freq
0         Coffee Shop 

#### Let's put that into a *pandas* dataframe
First, let's write a function to sort the venues in descending order.

In [76]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [116]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Restaurant,Café,Bar,Thai Restaurant,Sushi Restaurant,Asian Restaurant,Lounge,Breakfast Spot,Burger Joint
1,Berczy Park,Coffee Shop,Seafood Restaurant,Café,Beer Bar,Cheese Shop,Farmers Market,French Restaurant,Cocktail Bar,Restaurant,Bakery
2,"Brockton, Exhibition Place, Parkdale Village",Café,Breakfast Spot,Bakery,Coffee Shop,Convenience Store,Furniture / Home Store,Restaurant,Japanese Restaurant,Stadium,Italian Restaurant
3,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Yoga Studio,Spa,Auto Workshop,Brewery,Burrito Place,Comic Shop,Farmers Market,Fast Food Restaurant,Garden
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Service,Airport Lounge,Airport Terminal,Plane,Bar,Coffee Shop,Rental Car Location,Sculpture Garden,Boat or Ferry,Boutique


### Part 3.3 Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

In [114]:
# set number of clusters
kclusters = 4

Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(init = 'k-means++', n_clusters=kclusters, random_state=0, n_init =12).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[:] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0,
       2, 0, 0, 0, 0, 0, 2, 1, 0, 0, 0, 0, 0, 0, 3, 0, 0])

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [117]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
                               
toronto_merged = df_neighbor_toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160,0,Restaurant,Trail,Tennis Court,Playground,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
1,M5N,Central Toronto,Roselawn,43.711695,-79.416936,1,Garden,Yoga Studio,Dim Sum Restaurant,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
2,M4S,Central Toronto,Davisville,43.704324,-79.388790,0,Sandwich Place,Dessert Shop,Sushi Restaurant,Pizza Place,Café,Coffee Shop,Italian Restaurant,Gym,Japanese Restaurant,Farmers Market
3,M5P,Central Toronto,"Forest Hill North, Forest Hill West",43.696948,-79.411307,2,Jewelry Store,Trail,Park,Bus Line,Sushi Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Yoga Studio
4,M5R,Central Toronto,"The Annex, North Midtown, Yorkville",43.672710,-79.405678,0,Coffee Shop,Café,Sandwich Place,Pizza Place,BBQ Joint,Pharmacy,Cosmetics Shop,Pub,Middle Eastern Restaurant,Burger Joint


In [118]:
print('check if there is any N/A entries', toronto_merged.isnull().values.sum())
toronto_merged.dropna(inplace = True)

check if there is any N/A entries 0


In [119]:
print('check if there is any N/A entries', toronto_merged.isnull().values.sum())
toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].astype('int')

check if there is any N/A entries 0


#### Finally, let's visualize the resulting clusters

In [120]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' (Cluster ' + str(cluster) +')', parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Clusters
Examine each cluster and determine the discriminating venue categories that distinguish each cluster.

#### Cluster 0

In [121]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,0,Restaurant,Trail,Tennis Court,Playground,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
2,Central Toronto,0,Sandwich Place,Dessert Shop,Sushi Restaurant,Pizza Place,Café,Coffee Shop,Italian Restaurant,Gym,Japanese Restaurant,Farmers Market
4,Central Toronto,0,Coffee Shop,Café,Sandwich Place,Pizza Place,BBQ Joint,Pharmacy,Cosmetics Shop,Pub,Middle Eastern Restaurant,Burger Joint
6,Central Toronto,0,Hotel,Gym,Breakfast Spot,Sandwich Place,Park,Department Store,Food & Drink Shop,Distribution Center,Dog Run,Doner Restaurant
7,Central Toronto,0,Clothing Store,Coffee Shop,Sporting Goods Shop,Mexican Restaurant,Diner,Miscellaneous Shop,Dessert Shop,Park,Bagel Shop,Fast Food Restaurant
8,Central Toronto,0,Pub,Coffee Shop,Pizza Place,Liquor Store,Sports Bar,Restaurant,Supermarket,Sushi Restaurant,Light Rail Station,Fried Chicken Joint
9,Downtown Toronto,0,Coffee Shop,Café,Hotel,Restaurant,Bar,Bakery,Seafood Restaurant,Gastropub,American Restaurant,Italian Restaurant
10,Downtown Toronto,0,Coffee Shop,Café,Restaurant,Hotel,Clothing Store,Cosmetics Shop,Beer Bar,Italian Restaurant,Bakery,Diner
12,Downtown Toronto,0,Coffee Shop,Japanese Restaurant,Gay Bar,Restaurant,Sushi Restaurant,Café,Fast Food Restaurant,Hotel,Mediterranean Restaurant,Men's Store
13,Downtown Toronto,0,Coffee Shop,Café,Park,Bakery,Pub,Mexican Restaurant,Breakfast Spot,Restaurant,Theater,Event Space


#### Cluster 1

In [122]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Central Toronto,1,Garden,Yoga Studio,Dim Sum Restaurant,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


#### Cluster 2

In [123]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Central Toronto,2,Jewelry Store,Trail,Park,Bus Line,Sushi Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Yoga Studio
5,Central Toronto,2,Park,Swim School,Bus Line,Yoga Studio,Diner,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
11,Downtown Toronto,2,Park,Playground,Trail,Yoga Studio,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant


#### Cluster 3

In [124]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
28,East Toronto,3,Pub,Coffee Shop,Health Food Store,Trail,Yoga Studio,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop


#### Cluster 4

In [125]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
